In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving IMDB Dataset.csv - IMDB Dataset.csv to IMDB Dataset.csv - IMDB Dataset.csv


In [ ]:

data = pd.read_csv('IMDB Dataset.csv - IMDB Dataset.csv')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopwords = set(stopwords.words('english'))
filtered_words = [word for word in data['review'] if word not in stopwords]
print(filtered_words)

Buffered data was truncated after reaching the output size limit.

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.tail(20)

,review,sentiment
49980,A stunning film of high quality.<br /><br />Ap...,positive
49981,"And I repeat, please do not see this movie! Th...",negative
49982,"To be hones, I used to like this show and watc...",negative
49983,"I loved it, having been a fan of the original ...",positive
49984,Hello it is I Derrick Cannon and I welcome you...,negative
49985,Imaginary Heroes is clearly the best film of t...,positive
49986,This movie is a disgrace to the Major League F...,negative
49987,A remake of Alejandro Amenabar's Abre los Ojos...,negative
49988,"When I first tuned in on this morning news, I ...",negative
49989,I got this one a few weeks ago and love it! It...,positive


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data.replace({'sentiment': {'positive':1, 'negative':0}}, inplace=True)

<ipython-input-13-db273a0c1f24>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment': {'positive':1, 'negative':0}}, inplace=True)


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [ ]:
print(train_data['sentiment'].value_counts())
print(test_data['sentiment'].value_counts())

sentiment
0    20039
1    19961
Name: count, dtype: int64
sentiment
1    5039
0    4961
Name: count, dtype: int64


In [ ]:
print(train_data.head())
print(test_data.head())

                                                  review  sentiment
39087  That's what I kept asking myself during the ma...          0
30893  I did not watch the entire movie. I could not ...          0
45278  A touching love story reminiscent of In the M...          1
16398  This latter-day Fulci schlocker is a totally a...          0
13653  First of all, I firmly believe that Norwegian ...          0
                                                  review  sentiment
33553  I really liked this Summerslam due to the look...          1
9427   Not many television shows appeal to quite as m...          1
199    The film quickly gets to a major chase scene w...          0
12447  Jane Austen would definitely approve of this o...          1
39489  Expectations were somewhat high for me when I ...          0


In [ ]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=100)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=100)

In [ ]:
print(X_train)

[[    5    26    39 ...   205   351  3857]
 [  102  4701     2 ...    89   103     9]
 [    0     0     0 ...     2   710    62]
 ...
 [    0     0     0 ...  1641     2   603]
 [    0     0     0 ...   125  7284 16290]
 [    8    11    17 ...    70    73  2062]]


In [ ]:
print(X_test)

[[14258  1158  1527 ...   995   719   155]
 [  600     5    64 ...     7  9049  8492]
 [20195  2082  2541 ...    50  1088    96]
 ...
 [ 4097 12655   607 ...   125   200  3241]
 [    6   268     8 ...  1066     1  2304]
 [   63  2471     5 ...     1   332    27]]


In [ ]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [ ]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=50000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.build(input_shape=(None, 200))

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 200, 128)       │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,531,713 (24.92 MB)

 Trainable params: 6,531,713 (24.92 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 181s 363ms/step - accuracy: 0.7756 - loss: 0.4775 - val_accuracy: 0.8435 - val_loss: 0.3568
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 341ms/step - accuracy: 0.8988 - loss: 0.2600 - val_accuracy: 0.8500 - val_loss: 0.3448
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 339ms/step - accuracy: 0.9350 - loss: 0.1749 - val_accuracy: 0.8490 - val_loss: 0.3740
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 341ms/step - accuracy: 0.9612 - loss: 0.1149 - val_accuracy: 0.8478 - val_loss: 0.4198
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 347ms/step - accuracy: 0.9737 - loss: 0.0789 - val_accuracy: 0.8424 - val_loss: 0.4779


In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.8495 - loss: 0.4781
Test loss: 0.48076918721199036
Test accuracy: 0.847100019454956


In [ ]:
Y_pred = (model.predict(X_test) >= 0.5).astype(int)
cm = confusion_matrix(Y_test, Y_pred)
print(cm)


313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 40ms/step
[[4176  785]
 [ 744 4295]]


In [ ]:
report = classification_report(Y_test, Y_pred, target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.85      0.84      0.85      4961
    Positive       0.85      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [ ]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = 'Positive' if prediction[0][0] >= 0.5 else 'Negative'
  return sentiment

In [ ]:
new_review = 'This movie was fantastic! I loved every minute of it.'
predicted_sentiment = predict_sentiment(new_review)
print(f'Predicted sentiment: {predicted_sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
Predicted sentiment: Positive


In [ ]:
new_review = 'This Movie was not that good'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
The sentiment of the review is: Negative


In [ ]:
new_review = 'This movie was ok but not that good.'
sentiment = predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
The sentiment of the review is: Negative
